In [1]:
from lca_conf import SpotLCA
from RunDEMC.io import load_results
from RunDEMC import Model, Param, dists
from pathlib2 import Path
import pandas as pd
import numpy as np

In [2]:
SCORE_MIN_RT = 0.35
SCORE_MAX_RT = 1.35
def flkr_score(correct, rts):
    accuracy = (correct.astype(bool).mean() - 0.5) / 0.5
    rts = (rts - rts.min()) / rts.max() + 0.35
    speeds = (np.log(SCORE_MAX_RT + 1) - np.log(rts + 1)) / (np.log(SCORE_MAX_RT + 1) - np.log(SCORE_MIN_RT + 1))
    speed = speeds.mean()
    score = speed * accuracy * 100
    return score

In [3]:
subid = '11nuj5ty67ojohm39cmzbt23'
csv_dir = Path('/cogmood/data/pilot/to_model')
res_dir = Path('/cogmood/data/derivatives/model_res/')

In [4]:
sub_ids = [fp.parts[-1].split('.')[0].split('-')[-1] for fp in sorted(csv_dir.glob('flkr-*.csv'))]

In [5]:
log_shift = .05
max_rt = 2
burnin = 400
conditions = ['+', '=', '~']
nsims = 10000
p_post_points = 1000

def_params = dict(r=.1,
                  p=1.0,
                  sd0=2.0,
                  bin_lim=.5,
                  in_bin=1.5,
                  out_bin=2.5,
                  sd_min=.01,
                  K=.1,
                  L=.5,
                  U=0.0,
                  eta=1.,
                  t0=.25,
                  thresh=1.,
                  alpha=0.,
                  max_time=5.0,
                  truncate=True,
                  dt=.01, tau=.1)


In [ ]:
all_dat_scores = []
all_map_scores = []
ppres = []
for subid in sub_ids:
    print subid
    s = csv_dir / ('flkr-' + subid + '.csv')

    dat = pd.read_csv(s)
    ddat = {}
    for c in conditions:
        ind = (dat.condition == c) & (dat.rt < max_rt)
        d = {'rt':np.log(np.array(dat[ind].rt)+log_shift),
             'resp': np.array(~dat[ind]['correct'], dtype=np.int)}
        ddat[c] = d

    mdl_tgz = res_dir / ('flanker_flkr-' + subid + '.tgz')
    res = load_results(mdl_tgz.as_posix())
    
    # calculate scores on real data
    dat_scores = {'sub_id':subid}
    dat_scores['total'] = flkr_score(dat.correct.astype(bool).values, dat.rt.values)
    for c in ['+', '=', '~']:
        ind = (dat.condition == c) & (dat.rt < max_rt)
        d = {'rt':np.log(np.array(dat[ind].rt)+log_shift),
             'resp': np.array(~dat[ind]['correct'], dtype=np.int)}
        ddat[c] = d
        correct = ~(d['resp'].astype(bool))
        rts = dat[ind].rt.values
        dat_scores[c] = flkr_score(correct, rts)
    all_dat_scores.append(dat_scores)
    
    # calculate scores on map parameters
    best_ind = res['weights'][burnin:].argmax()
    indiv = [res['particles'][burnin:, :, i].ravel()[best_ind]
            for i in range(res['particles'].shape[-1])]
    best_ps = {p:v for p, v in zip(res['param_names'], indiv)}
    params = best_ps
    
    mod_params = def_params.copy()
    mod_params.update(params)
    
    
    dbin = {}
    dbin['+'] = {
             'bins': np.array([[-(mod_params['out_bin']), -(mod_params['in_bin'])],
                          [-(mod_params['in_bin']), -(mod_params['bin_lim'])],
                          [-(mod_params['bin_lim']), mod_params['bin_lim']],
                          [mod_params['bin_lim'], mod_params['in_bin']],
                          [mod_params['in_bin'], mod_params['out_bin']]], dtype=np.float32),
             'bin_ind': np.array([0,0,0,0,0], dtype=np.int32),
             'nbins': 5}
    dbin['='] = {
             'bins': np.array([[-(mod_params['out_bin']), -(mod_params['in_bin'])],
                          [-(mod_params['in_bin']), -(mod_params['bin_lim'])],
                          [-(mod_params['bin_lim']), mod_params['bin_lim']],
                          [mod_params['bin_lim'], mod_params['in_bin']],
                          [mod_params['in_bin'], mod_params['out_bin']]], dtype=np.float32),
             'bin_ind': np.array([1,0,0,0,1], dtype=np.int32),
             'nbins': 5}
    dbin['~'] = {
             'bins': np.array([[-(mod_params['out_bin']), -(mod_params['in_bin'])],
                          [-(mod_params['in_bin']), -(mod_params['bin_lim'])],
                          [-(mod_params['bin_lim']), mod_params['bin_lim']],
                          [mod_params['bin_lim'], mod_params['in_bin']],
                          [mod_params['in_bin'], mod_params['out_bin']]], dtype=np.float32),
             'bin_ind': np.array([0,1,0,1,0], dtype=np.int32),
             'nbins': 5}

    mod_params['x_init'] = np.ones(len(dbin), dtype=np.float32)*(mod_params['thresh']*float(1/3.))
    
    out_times = {'total':[]}
    corrects = {'total': []}
    map_scores = {'sub_id': subid}
    for c in conditions:
        lca = SpotLCA(nitems=2, nbins=dbin[c]['nbins'],
                      nsims=nsims, log_shift=log_shift, nreps=1)
        mod_params['bins'] = dbin[c]['bins']
        mod_params['bin_ind'] = dbin[c]['bin_ind']
        out_time, x_ind, x_out, conf = lca.simulate(**mod_params)
        x_correct = ~((x_ind == 0).astype(bool))
        out_times[c] = out_time
        out_times['total'].extend(list(out_time))
        corrects[c] = x_correct
        corrects['total'].extend(list(x_correct))
        map_scores[c] = flkr_score(x_correct, out_time)
    map_scores['total'] = flkr_score(np.array(corrects['total']), np.array(out_times['total']))
    all_map_scores.append(map_scores)
    
    for ppix in range(p_post_points):
        if ppix %100 == 0:
            print ppix
        particles = res['particles'][burnin:]
        n_chains = particles.shape[1]
        n_draws = particles.shape[0]
        part_id = np.random.choice(range(n_chains * n_draws))
        params = {p:particles[:, :, i].ravel()[part_id] for i,p in enumerate(res['param_names'])}

        mod_params = def_params.copy()
        mod_params.update(params)
        mod_params['x_init'] = np.ones(len(dbin), dtype=np.float32)*(mod_params['thresh']*float(1/3.))

        out_times = {'total':[]}
        corrects = {'total': []}
        sim_scores = {'ppix':ppix, 'part_id':part_id, 'sub_id':subid}
        for c in conditions:
            lca = SpotLCA(nitems=2, nbins=dbin[c]['nbins'],
                          nsims=nsims, log_shift=log_shift, nreps=1)
            mod_params['bins'] = dbin[c]['bins']
            mod_params['bin_ind'] = dbin[c]['bin_ind']
            out_time, x_ind, x_out, conf = lca.simulate(**mod_params)
            x_correct = ~((x_ind == 0).astype(bool))
            out_times[c] = out_time
            out_times['total'].extend(list(out_time))
            corrects[c] = x_correct
            corrects['total'].extend(list(x_correct))
            sim_scores[c] = flkr_score(x_correct, out_time)
        sim_scores['total'] = flkr_score(np.array(corrects['total']), np.array(out_times['total']))
        ppres.append(sim_scores)

11nuj5ty67ojohm39cmzbt23
0
100
200
300


In [133]:
out_times = {'total':[]}
corrects = {'total': []}
map_scores = {'sub_id': subid}
for c in conditions:
    lca = SpotLCA(nitems=2, nbins=dbin[c]['nbins'],
                  nsims=nsims, log_shift=log_shift, nreps=1)
    mod_params['bins'] = dbin[c]['bins']
    mod_params['bin_ind'] = dbin[c]['bin_ind']
    out_time, x_ind, x_out, conf = lca.simulate(**mod_params)
    x_correct = ~(x_ind.astype(bool))
    out_times[c] = out_time
    out_times['total'].extend(list(out_time))
    corrects[c] = x_correct
    corrects['total'].extend(list(x_correct))
    map_scores[c] = flkr_score(x_correct, out_time)
map_scores['total'] = flkr_score(np.array(corrects['total']), np.array(out_times['total']))
map_scores

{'+': 79.83615989685057,
 '=': 80.82382593154907,
 'sub_id': '11nuj5ty67ojohm39cmzbt23',
 'total': 80.70475405772525,
 '~': 77.99508069992066}

In [115]:
ppres = []
for ppix in range(p_post_points):
    particles = res['particles'][burnin:]
    n_chains = particles.shape[1]
    n_draws = particles.shape[0]
    part_id = np.random.choice(range(n_chains * n_draws))
    params = {p:particles[:, :, i].ravel()[part_id] for i,p in enumerate(res['param_names'])}
    
    mod_params = def_params.copy()
    mod_params.update(params)
    mod_params['x_init'] = np.ones(len(dbin), dtype=np.float32)*(mod_params['thresh']*float(1/3.))
    
    out_times = {'total':[]}
    corrects = {'total': []}
    sim_scores = {'ppix':ppix, 'part_id':part_id, 'sub_id':sub_id}
    for c in conditions:
        lca = SpotLCA(nitems=2, nbins=dbin[c]['nbins'],
                      nsims=nsims, log_shift=log_shift, nreps=1)
        mod_params['bins'] = dbin[c]['bins']
        mod_params['bin_ind'] = dbin[c]['bin_ind']
        out_time, x_ind, x_out, conf = lca.simulate(**mod_params)
        x_correct = ~(x_ind.astype(bool))
        out_times[c] = out_time
        out_times['total'].extend(list(out_time))
        corrects[c] = x_correct
        corrects['total'].extend(list(x_correct))
        sim_scores[c] = flkr_score(x_correct, out_time)
    sim_scores['total'] = flkr_score(np.array(corrects['total']), np.array(out_times['total']))
    ppres.append(sim_scores)
ppres = pd.DataFrame(ppres)

KeyboardInterrupt: 

In [117]:
ppres = pd.DataFrame(ppres)

In [123]:
(ppres['+'] > dat_scores['+']).mean()

0.007662835249042145

In [124]:
(ppres['='] > dat_scores['=']).mean()

0.29118773946360155

In [125]:
(ppres['~'] > dat_scores['~']).mean()

0.7854406130268199

In [126]:
(ppres['total'] > dat_scores['total']).mean()

0.10344827586206896